## 50.i Forecasting - Facebook Prophet
</font> [Source: Facebook GitHub](https://facebook.github.io/prophet/)

* [Prophet (FB Blog)-forecasting at scale](https://research.fb.com/blog/2017/02/prophet-forecasting-at-scale/)
* [Forecasting at Scale](https://peerj.com/preprints/3190.pdf)

In [1]:
# required packages
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)
import plotly.graph_objects as go
#plot the graph using plolty
import plotly.offline as py
from fbprophet.plot import plot_plotly
from fbprophet import Prophet 
from fbprophet.plot import plot_cross_validation_metric
#The style package adds support for easy-to-switch plotting "styles" with 
## the same parameters as a matplotlib rc file (which is read at startup to configure matplotlib).
%matplotlib inline
plt.style.use('ggplot')

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
#attention might have problems with holiday package, 
#downgrate holidays via: pip install 'holidays==0.9.12'

## 50.ii Trivial Forecast_Rolling mean

In [ ]:
# generate an input df
df = pd.DataFrame({'X': np.arange(0,10)})
# take the window of 3 and write the average as y named column
df['y']=df.rolling(3).mean() 

In [ ]:
df.head()

In [ ]:
# Import CSV file as dataframe
df_all = pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';')
# select the data of only one country, here: germany 
df=df_all[['date','Germany']]
# rename the column name 
df=df.rename(columns={'date': 'ds','Germany': 'y'})
df.head()

In [ ]:
ax = df.set_index('ds').plot(figsize=(12, 8), logy=True)
ax.set_ylabel('Confimed cases per day')
ax.set_xlabel('Timeline in Days')
plt.show()

In [ ]:
# set the uncertainty interval to 95% (the Prophet default is 80%)
#model = Prophet(interval_width=0.95) # for piecwise linear model
model = Prophet(growth='logistic')   # for logistic model

In [ ]:
# the column 'cap' is only mandatory for the logistic model
df['cap']=1000000.
model.fit(df)

In [ ]:
# define the periods and the frequency 'D'== days
future_dates = model.make_future_dataframe(periods=7, freq='D')
future_dates['cap']=1000000. # only mandatory for the logistic model
future_dates.tail()

In [ ]:
# predict according to the scikit-learn standard
forecast = model.predict(future_dates)

In [ ]:
model.plot(forecast,uncertainty=True ); # fbprohet is also responisble for rendering the output

In [ ]:
fig = plot_plotly(model, forecast)  
fig.update_layout(width=1024,height=800,xaxis_title="Time",yaxis_title="Confirmed infected people from johns hopkins csse, log-scale)",)
fig.update_yaxes(type="log",range=[1.1,5.5])
py.iplot(fig)

In [ ]:
# sorting the values according to column named ds
forecast.sort_values(by='ds').head()

In [ ]:
#plot the graphs
model.plot_components(forecast);

In [ ]:
forecast[['ds','trend']].set_index('ds').plot(figsize=(12, 8),logy=True)

## 50.iii Cross_Validation

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model, 
                         initial='40 days', # we take the first 30 days for training
                         period='1 days',  # every  days a new prediction run
                         horizon = '7 days') #we predict 7days into the future

In [ ]:
df_cv.sort_values(by=['cutoff','ds'])[0:12]
df_cv.head()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)

### 50.iii.a. Performance matrix

In [ ]:
# the performance matrix shows the result for all horizon
df_p

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mape',)

## 50.iv Diagonal_plot 

In [ ]:
horizon='7 days'
df_cv['horizon']=df_cv.ds-df_cv.cutoff

date_vec=df_cv[df_cv['horizon']==horizon]['ds']
y_hat=df_cv[df_cv['horizon']==horizon]['yhat']
y=df_cv[df_cv['horizon']==horizon]['y']

In [ ]:
df_cv_7=df_cv[df_cv['horizon']==horizon]
df_cv_7.tail()

In [ ]:
type(df_cv['horizon'][0])

In [ ]:
# plotting the diagonal plot
fig, ax = plt.subplots(1, 1)
ax.plot(np.arange(max(y)),np.arange(max(y)),'--',label='diagonal')
ax.plot(y,y_hat,'-',label=horizon)  # horizon is a np.timedelta objct

ax.set_title('Diagonal Plot of forecasting')
ax.set_ylim(10, max(y))
ax.set_xlabel('Truth: y')
ax.set_ylabel('Prediciton: y_hat')
ax.set_yscale('log')
ax.set_xlim(10, max(y))
ax.set_xscale('log')
ax.legend(loc='best',prop={'size': 16});

## 50.v. Trivial Forecast

In [ ]:
#function for mean absolute percentage error
def MAPE(y_true, y_pred): 
    ''' MAPE calculation '''
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
parse_dates=['date']
df_all = pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',parse_dates=parse_dates)
df_trivial=df_all[['date','Germany']]
df_trivial=df_trivial.rename(columns={'date': 'ds','Germany': 'y'})

### 50.v.a. Standard forecast 

In [ ]:
df_trivial['y_mean_r3']=df_trivial.y.rolling(3).mean() # take the average of 3 days

In [ ]:
# the result has to be shifted according to the prediciton horizon (here 7 days)
df_trivial['cutoff']=df_trivial['ds'].shift(7)
df_trivial['y_hat']=df_trivial['y_mean_r3'].shift(7)
df_trivial['horizon']=df_trivial['ds']-df_trivial['cutoff']
print('MAPE: '+str(MAPE(df_trivial['y_hat'].iloc[12:,], df_trivial['y'].iloc[12:,])))
df_trivial